### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
from tqdm import tqdm
import tests as t
import pickle
from tqdm import tqdm
import time

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [5]:
all_recs
users_with_all_recs = { key:value for (key,value)
                       in all_recs.items() if len(value) >= 10 }.keys()
# users_with_all_recs

In [6]:
 # Store user ids who have all their recommendations in this (10 or more)
users_who_need_recs = { key:value for (key,value)
                       in all_recs.items() if len(value) < 10 }.keys()
# Store users who still need recommendations here


In [7]:
# users_with_all_recs
# users_who_need_recs

In [8]:
len(users_with_all_recs)

22187

In [9]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [10]:
reviews.head(3)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [11]:
# create a dataframe similar to reviews, but ranked by rating for each user
ranked_df = reviews.sort_values(['user_id', 'rating'], ascending = False)

ranked_df["movie_rank"] = ranked_df.groupby("user_id")["rating"].rank("dense", ascending=False)
# ranked_df[ranked_df['user_id'] == 2]

### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [in the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar, and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (and will also be the number of 1's in the movie row within the orginal movie content matrix).

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [12]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
movie_content = movies.drop(['movie', 'genre', 'date'], axis = 1)
movie_content.set_index('movie_id', inplace = True)

# Take the dot product to obtain a movie x movie matrix of similarities
dot_prod_movies = movie_content.dot(movie_content.transpose())

In [14]:
dot_prod_movies.head(3)

movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
movie_id,,,,,,,,,,,,,,,,,,,,,
8,3,3,3,1,2,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
10,3,3,3,1,2,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
12,3,3,3,1,2,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0


In [23]:
np.max(dot_prod_movies[0])

3

In [72]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 31245
assert dot_prod_movies.shape[1] == 31245
# assert dot_prod_movies[0][0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [16]:
movies.head(3)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [68]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    # find the row of each movie id
    
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    similar_movies = dot_prod_movies[movie_id].reset_index().sort_values(by = 'movie_id',
                                                                         ascending = False)
    similar_movies = similar_movies[
        similar_movies[movie_id] == similar_movies[movie_id].max()]
    
    # pull the movie titles based on the indices
    similar_movies = get_movie_names(similar_movies['movie_id'])
    
    return similar_movies
    
# You made this function in an earlier notebook - using again here    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
   
    return movie_lst


def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    recs = {}
    # How many users for progress bar
    num_of_users_to_rec = len(users_who_need_recs)
    keys = list(users_who_need_recs)
    
    for i in tqdm (range (num_of_users_to_rec), desc="Loading..."):
        user_id = keys[i]
        movie_id = ranked_df[(ranked_df['user_id']==user_id) &
                             (ranked_df['movie_rank']==1)]['movie_id'].values[0]
        recs[user_id] = find_similar_movies(movie_id)
    
    # Create the progressbar

    
    # For each user
        
        # Update the progress bar


        # Pull only the reviews the user has seen

        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user



    # finish the progress bar

    
    return recs

In [69]:
recs = make_recs()

Loading...: 100%|██████████| 1325/1325 [00:50<00:00, 26.35it/s] 


### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [71]:
# Explore recommendations
recs[26]

["Don's Plum (2001)",
 'State and Main (2000)',
 'Sånger från andra våningen (2000)',
 'Return to Me (2000)',
 'Bruno (2000)',
 'Committed (2000)',
 'High Fidelity (2000)',
 'The Next Best Thing (2000)',
 'Ghost World (2001)',
 'Hanging Up (2000)',
 'Interstate 60: Episodes of the Road (2002)',
 'Memórias Póstumas (2001)',
 'Keeping the Faith (2000)',
 'Nurse Betty (2000)',
 'Sunset Strip (2000)',
 'Almost Famous (2000)',
 'Wonder Boys (2000)',
 'Down to You (2000)',
 'Pantaleón y las visitadoras (2000)',
 'The Fighting Temptations (2003)',
 '28 Days (2000)',
 'Badding (2000)',
 'Panic (2000)',
 'Where the Heart Is (2000)',
 'Riding in Cars with Boys (2001)',
 'Trasgredire (2000)',
 'Beautiful Joe (2000)',
 'Coyote Ugly (2000)',
 'Rock Star (2001)',
 'Two Family House (2000)',
 'Tillsammans (2000)',
 'Boys and Girls (2000)',
 'Dr. T and the Women (2000)',
 'Thomas and the Magic Railroad (2000)',
 'AmnesiA (2001)',
 'The Dish (2000)',
 'My First Mister (2001)',
 "Was tun, wenn's brennt?

In [73]:
# Some characteristics of my content based recommendations
users_without_all_recs = { key:value for (key,value)
                       in recs.items() if len(value) < 10 }.keys()#store user ids without recs
users_with_all_recs = { key:value for (key,value)
                       in recs.items() if len(value) > 10 }.keys()# users with all recs here
no_recs = { key:value for (key,value)
                       in recs.items() if len(value) == 0 }.keys()# users with no recommendations

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 103 users without all 10 recommendations we would have liked to have.
There were 1216 users with all 10 recommendations we would like them to have.
There were 0 users with no recommendations at all!


In [2]:
user_items = reviews[['user_id', 'movie_id', 'rating']]

In [3]:
# Closer look at individual user characteristics - this may help it was from an earlier notebook

user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [4]:
def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


movies_watched(2)

array([ 422720,  454876,  790636,  816711, 1091191, 1103275, 1322269,
       1390411, 1398426, 1431045, 1433811, 1454468, 1535109, 1675434,
       1798709, 2017038, 2024544, 2294629, 2361509, 2381249, 2726560,
       2883512, 3079380], dtype=int64)

In [74]:
# More exploring
print("Some of the movie lists for users without any recommendations include:")
# Just one of many questions you could continue exploring...

Some of the movie lists for users without any recommendations include:


### Now What?  

Well, if you were really strict with your criteria for how similar two movies are (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!

In [ ]:
# Cells for exploring